# Radiomics Feature Extraction

Extracting features from PET and CT images.

In [1]:
import os
import nrrd
import utils 
import radiomics

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from feature_extraction import feature_extractor

%matplotlib inline

In [2]:
path_masksdir = './../../data_source/images/masks_nrrd/'
path_ct_imagedir = './../../data_source/images/ct_nrrd/'
path_pet_imagedir = './../../data_source/images/pet_nrrd/'

In [3]:
paths_ct_images = utils.sample_paths(
    path_ct_imagedir, path_masksdir, target_format='nrrd'
)
paths_pet_images = utils.sample_paths(
    path_pet_imagedir, path_masksdir, target_format='nrrd'
)

## Setup

In [ ]:
def z_score_transform(image, n=None):
    # Texture features is shown to be more stable after GL normalizations.
    # Ref: Voxel size and gray level normalization of CT radiomic features 
    #      in lung cancer.
    _image = np.copy(image)
    
    # Z-score transformation.
    image_std = np.std(_image)
    image_zscore = (_image - np.mean(_image)) / image_std
    
    # Remove outliers.
    if n is not None:
        outliers = n * image_std 
        image_zscore = np.clip(image_zscore, -1.0 * outliers, outliers)

    return image_zscore


def bin_widths(path_images, nbins, n=3, z_scoring=False, outlier_removal=None):
    """Average min and max to compare histogram across stacks.
    Fixed bin iwdth (and not fixed bin size) to compare texture
    features across stacks.
    
    """
    img_max, img_min = [], []
    for image_path in path_images:
        image, _ = nrrd.read(image_path['Image'])
        
        if z_scoring:
            image = z_score_transform(image, n=outlier_removal)
        
        img_max.append(np.ceil(np.max(image)))
        img_min.append(np.floor(np.min(image)))
  
    return (np.mean(img_max) - np.mean(img_min)) / nbins

In [ ]:
bin_widths(paths_ct_images, 32), bin_widths(paths_ct_images, 64), bin_widths(paths_ct_images, 128)

In [ ]:
bin_widths(paths_pet_images, 32), bin_widths(paths_pet_images, 64), bin_widths(paths_pet_images, 128)

In [ ]:
# NOTE: Bin iwdths for Z-scored CT texture features.
print(bin_widths(paths_ct_images, 32, z_scoring=True, outlier_removal=3))
print(bin_widths(paths_ct_images, 64, z_scoring=True, outlier_removal=3))
print(bin_widths(paths_ct_images, 128, z_scoring=True, outlier_removal=3))

## Calculate features

In [9]:
ct_param_files = [
    #'./parameter_files/shape_original_images/shape_config.yaml',
    #'./parameter_files/firstorder_original_images/ct32_firstorder_original_images_config.yaml',
    #'./parameter_files/firstorder_original_images/ct64_firstorder_original_images_config.yaml',
    #'./parameter_files/firstorder_original_images/ct128_firstorder_original_images_config.yaml'
    './parameter_files/texture_original_images/ct32_texture_original_images_config.yaml',
    './parameter_files/texture_original_images/ct64_texture_original_images_config.yaml',
    './parameter_files/texture_original_images/ct128_texture_original_images_config.yaml',
]
ct_feature_files = [
    #'./../../data_source/radiomic_features/shape_original_images/shape_original_masks.csv',
    #'./../../data_source/radiomic_features/firstorder_original_images/ct32_firstorder_original_images.csv',
    #'./../../data_source/radiomic_features/firstorder_original_images/ct64_firstorder_original_images.csv',
    #'./../../data_source/radiomic_features/firstorder_original_images/ct128_firstorder_original_images.csv'
    './../../data_source/radiomic_features/texture_original_images/ct32_texture_original_images.csv',
    './../../data_source/radiomic_features/texture_original_images/ct64_texture_original_images.csv',
    './../../data_source/radiomic_features/texture_original_images/ct128_texture_original_images.csv',
]
for ct_param_file, ct_feature_file in zip(ct_param_files, ct_feature_files):
    feature_extractor(
        param_file=ct_param_file, 
        paths_to_images_and_masks=paths_ct_images, 
        verbose=1, 
        path_to_results=ct_feature_file,
        n_jobs=None, 
        drop_missing=True, 
        variance_thresh=0.0
    )

Initiated feature extraction.


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    8.8s
[Parallel(n_jobs=3)]: Done 198 out of 198 | elapsed:   33.8s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Initiated feature extraction.


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    8.3s
[Parallel(n_jobs=3)]: Done 198 out of 198 | elapsed:   31.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Initiated feature extraction.


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    7.6s
[Parallel(n_jobs=3)]: Done 198 out of 198 | elapsed:   41.7s finished


In [10]:
pet_param_files = [
    #'./parameter_files/firstorder_original_images/pet32_firstorder_original_images_config.yaml',
    #'./parameter_files/firstorder_original_images/pet64_firstorder_original_images_config.yaml',
    #'./parameter_files/firstorder_original_images/pet128_firstorder_original_images_config.yaml'
    './parameter_files/texture_original_images/pet32_texture_original_images_config.yaml',
    './parameter_files/texture_original_images/pet64_texture_original_images_config.yaml',
    './parameter_files/texture_original_images/pet128_texture_original_images_config.yaml',
]
pet_feature_files = [
    #'./../../data_source/radiomic_features/firstorder_original_images/pet32_firstorder_original_images.csv',
    #'./../../data_source/radiomic_features/firstorder_original_images/pet64_firstorder_original_images.csv',
    #'./../../data_source/radiomic_features/firstorder_original_images/pet128_firstorder_original_images.csv'
    './../../data_source/radiomic_features/texture_original_images/pet32_texture_original_images.csv',
    './../../data_source/radiomic_features/texture_original_images/pet64_texture_original_images.csv',
    './../../data_source/radiomic_features/texture_original_images/pet128_texture_original_images.csv',
]
for pet_param_file, pet_feature_file in zip(pet_param_files, pet_feature_files):
    feature_extractor(
        param_file=pet_param_file, 
        paths_to_images_and_masks=paths_pet_images, 
        verbose=1, 
        path_to_results=pet_feature_file,
        n_jobs=None, 
        drop_missing=True, 
        variance_thresh=0.0
    )

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Initiated feature extraction.


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    8.0s
[Parallel(n_jobs=3)]: Done 198 out of 198 | elapsed:   31.8s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Initiated feature extraction.


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    8.6s
[Parallel(n_jobs=3)]: Done 198 out of 198 | elapsed:   40.5s finished


Initiated feature extraction.


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:   25.5s
[Parallel(n_jobs=3)]: Done 198 out of 198 | elapsed:  1.5min finished
